# எளிய பட வகைப்பாளர்

இந்த நோட்புக் ஒரு முன்பே பயிற்சி பெற்ற நரம்பியல் வலைப்பின்னலின் உதவியுடன் படங்களை வகைப்படுத்துவது எப்படி என்பதை காட்டுகிறது.

**நீங்கள் கற்றுக்கொள்ளப்போகிறீர்கள்:**
- முன்பே பயிற்சி பெற்ற மாதிரியை ஏற்றுவது மற்றும் பயன்படுத்துவது
- பட முன்னேற்பாடு
- படங்களில் கணிப்புகளை செய்வது
- நம்பகத்தன்மை மதிப்பீடுகளை புரிந்துகொள்வது

**பயன்பாட்டு வழக்கு:** படங்களில் பொருட்களை அடையாளம் காணுதல் (உதாரணமாக "பூனை", "நாய்", "கார்" போன்றவை) 

---


## படி 1: தேவையான நூலகங்களை இறக்குமதி செய்யவும்

நமக்கு தேவையான கருவிகளை இறக்குமதி செய்வோம். இவை அனைத்தையும் இன்னும் புரிந்துகொள்ளவில்லை என்றால் கவலைப்பட வேண்டாம்!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## படி 2: முன்பே பயிற்சி பெற்ற மாடலை ஏற்றவும்

நாம் **MobileNetV2**-ஐ பயன்படுத்தப் போகிறோம், இது ஏற்கனவே மில்லியன் கணக்கான படங்களின் மீது பயிற்சி பெற்ற நரம்பியல் வலைப்பின்னல்.

இதற்கு **Transfer Learning** என்று அழைக்கப்படுகிறது - மற்றொருவர் பயிற்சி செய்த மாடலை பயன்படுத்துவது!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## படி 3: உதவியாளர் செயல்பாடுகள்

நாம் உருவாக்கும் மாதிரிக்கு படங்களை ஏற்றவும் தயாரிக்கவும் உதவியாளர் செயல்பாடுகளை உருவாக்குவோம்.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## படி 4: மாதிரி படங்களில் சோதனை செய்யுங்கள்

இணையத்தில் இருந்து சில படங்களை வகைப்படுத்த முயற்சிப்போம்!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ஒவ்வொரு படத்தையும் வகைப்படுத்தவும்


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## படி 5: உங்கள் சொந்த படங்களை முயற்சிக்கவும்!

கீழே உள்ள URL ஐ மாற்றி, நீங்கள் வகைப்படுத்த விரும்பும் எந்த பட URL ஐ வேண்டுமானாலும் பயன்படுத்துங்கள்.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 என்ன நடந்தது?

1. **நாம் முன்பே பயிற்சி பெற்ற மாடலை ஏற்றினோம்** - MobileNetV2 மில்லியன் கணக்கான படங்களில் பயிற்சி பெற்றது  
2. **நாம் படங்களை முன் செயலாக்கினோம்** - மாடலுக்காக அவற்றை அளவிடவும் வடிவமைக்கவும் செய்தோம்  
3. **மாடல் கணிப்புகளை செய்தது** - 1000 பொருள் வகைகளுக்கான சாத்தியக்கூறுகளை வெளியிட்டது  
4. **நாம் முடிவுகளை டிகோட் செய்தோம்** - எண்களை மனிதர்களுக்குப் புரியக்கூடிய லேபிள்களாக மாற்றினோம்  

### நம்பகத்தன்மை மதிப்பீடுகளைப் புரிந்துகொள்வது

- **90-100%**: மிகவும் நம்பகமானது (கிட்டத்தட்ட சரியாக இருக்கும்)  
- **70-90%**: நம்பகமானது (சரியாக இருக்க வாய்ப்பு உள்ளது)  
- **50-70%**: சில அளவுக்கு நம்பகமானது (சரியாக இருக்கலாம்)  
- **50% க்கும் குறைவாக**: அதிக நம்பகத்தன்மை இல்லை (சந்தேகமாக உள்ளது)  

### ஏன் கணிப்புகள் தவறாக இருக்கலாம்?

- **அசாதாரண கோணம் அல்லது ஒளி** - மாடல் சாதாரண புகைப்படங்களில் பயிற்சி பெற்றது  
- **பல பொருட்கள்** - மாடல் ஒரு முக்கிய பொருளை எதிர்பார்க்கிறது  
- **அரிதான பொருட்கள்** - மாடல் 1000 வகைகளை மட்டுமே அறிகிறது  
- **குறைந்த தரமான படம்** - மங்கலான அல்லது பிக்சலேட்டான படங்கள் கடினமாக இருக்கும்  

---  


## 🚀 அடுத்த படிகள்

1. **வித்தியாசமான படங்களை முயற்சிக்கவும்:**
   - [Unsplash](https://unsplash.com) இல் படங்களை தேடுங்கள்
   - வலது கிளிக் → "Copy image address" செய்து URL ஐ பெறுங்கள்

2. **சோதனை செய்யுங்கள்:**
   - கலைப்படைப்புகளை பயன்படுத்தினால் என்ன நடக்கிறது?
   - வெவ்வேறு கோணங்களில் பொருட்களை அடையாளம் காண முடியுமா?
   - பல பொருட்களை ஒரே நேரத்தில் எப்படி கையாளுகிறது?

3. **மேலும் கற்றுக்கொள்ளுங்கள்:**
   - [கம்ப்யூட்டர் விஷன் பாடங்களை](../lessons/4-ComputerVision/README.md) ஆராயுங்கள்
   - உங்கள் சொந்த பட வகைப்பாளியை பயிற்சி செய்ய கற்றுக்கொள்ளுங்கள்
   - CNNs (Convolutional Neural Networks) எப்படி வேலை செய்கின்றன என்பதை புரிந்துகொள்ளுங்கள்

---

## 🎉 வாழ்த்துக்கள்!

நீங்கள் ஒரு முன்னணி நரம்பியல் வலைப்பின்னலின் உதவியுடன் ஒரு பட வகைப்பாளியை உருவாக்கியுள்ளீர்கள்!

இந்தzelfde தொழில்நுட்பம் பின்வரும் செயல்பாடுகளுக்கு சக்தி அளிக்கிறது:
- Google Photos (உங்கள் புகைப்படங்களை ஒழுங்குபடுத்துதல்)
- சுய இயக்க வாகனங்கள் (பொருட்களை அடையாளம் காணுதல்)
- மருத்துவ பரிசோதனை (X-rays பகுப்பாய்வு)
- தரக் கட்டுப்பாடு (குறைபாடுகளை கண்டறிதல்)

தொடர்ந்து ஆராய்ந்து கற்றுக்கொள்ளுங்கள்! 🚀



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியக்க மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
